In [1]:
# the codes in Module 10
# load data
import pandas as pd
train_data = pd.read_csv("C:/Users/sanketn/Documents/IU/Kaggle/train.csv")
test_data = pd.read_csv("C:/Users/sanketn/Documents/IU/Kaggle/test.csv")

In [2]:
# take target out of training set
Y = train_data['label']
train_data = train_data.drop(['label'], axis=1)

In [24]:
test_Y = Y[:8400]

In [3]:
train_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# remove the features that are all zeros
total_data = train_data
total_data = total_data.append(test_data)
for col in total_data.columns:
    if len(total_data[total_data[col] == 0]) == len(total_data[col]):
        train_data = train_data.drop([col], axis=1)
        test_data = test_data.drop([col], axis=1)

In [5]:
# min-max scaling
train_data = (train_data - train_data.min()) / (train_data.max() - train_data.min())
test_data = (test_data - test_data.min()) / (test_data.max() - test_data.min())

In [11]:
test = train_data[:8400]
test.shape[0]

8400

In [6]:
def countMissing(data):
    missing = data.columns[data.isnull().any()].tolist()
    return missing
misTrain = countMissing(train_data)
misTest = countMissing(test_data)
misTotal = list(set().union(misTrain, misTest))

In [7]:
def imputation(data, column, value):
    data.loc[data[column].isnull(), column] = value

In [17]:
print(misTrain, len(misTrain))

['pixel139', 'pixel141', 'pixel196', 'pixel392', 'pixel420', 'pixel421', 'pixel448', 'pixel532', 'pixel645', 'pixel731', 'pixel760'] 11


In [18]:
print(misTest, len(misTest))

['pixel12', 'pixel13', 'pixel14', 'pixel15', 'pixel32', 'pixel51', 'pixel58', 'pixel113', 'pixel364', 'pixel588', 'pixel615', 'pixel616', 'pixel643', 'pixel698', 'pixel702', 'pixel726', 'pixel753', 'pixel779'] 18


In [15]:
for ele in misTrain:
    imputation(train_data, ele, 1)

In [16]:
for ele in misTest:
    imputation(test_data, ele, 1)

In [19]:
# One-Vs-Rest with regularized logistic regression models
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
res = OneVsRestClassifier(linear_model.LogisticRegression(penalty = "l1")).fit(train_data, Y).predict(test_data)

In [20]:
# One-Vs-One with regularized logistic regression models
from sklearn.multiclass import OneVsOneClassifier
from sklearn import linear_model
res = OneVsOneClassifier(linear_model.LogisticRegression(penalty = "l1")).fit(train_data, Y).predict(test_data)

In [32]:
from sklearn import cross_validation
alphas = [0.01, 0.1, 1.0, 10]
regs = ["l1", "l2"]
scores = []
param = []
for alpha in alphas:
    for reg in regs:
        lm = OneVsRestClassifier(linear_model.LogisticRegression(penalty = reg, C = alpha))
        scores.append(cross_validation.cross_val_score(lm, train_data, Y, scoring="accuracy", cv = 2).mean())
        param.append([alpha, reg])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

    parameter     score
4   [1.0, l1]  0.911929
5   [1.0, l2]  0.910619
3   [0.1, l2]  0.910214
7    [10, l2]  0.904952
6    [10, l1]  0.902548
2   [0.1, l1]  0.902119
1  [0.01, l2]  0.898928
0  [0.01, l1]  0.852238


In [33]:
from sklearn import cross_validation
alphas = [0.01, 0.1, 1.0, 10]
regs = ["l1", "l2"]
scores = []
param = []
for alpha in alphas:
    for reg in regs:
        lm = OneVsOneClassifier(linear_model.LogisticRegression(penalty = reg, C = alpha))
        scores.append(cross_validation.cross_val_score(lm, train_data, Y, scoring="accuracy", cv = 2).mean())
        param.append([alpha, reg])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

    parameter     score
5   [1.0, l2]  0.934500
3   [0.1, l2]  0.933214
4   [1.0, l1]  0.932333
7    [10, l2]  0.927096
6    [10, l1]  0.922929
1  [0.01, l2]  0.918524
2   [0.1, l1]  0.914405
0  [0.01, l1]  0.836976


In [29]:
# One-Vs-Rest with regularized logistic regression models
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
onelm = OneVsRestClassifier(linear_model.LogisticRegression(penalty = "l1", C = 1.0)).fit(train_data, Y)
res = onelm.predict(test)

In [56]:
res1 = pd.Series(res)

In [55]:
from sklearn import metrics
print(metrics.accuracy_score(test_Y, res1))

0.93


In [61]:
# One-Vs-One with regularized logistic regression models
from sklearn.multiclass import OneVsOneClassifier
from sklearn import linear_model
res = OneVsOneClassifier(linear_model.LogisticRegression(penalty = "l2", C = 0.3)).fit(train_data, Y).predict(test)

In [62]:
res1 = pd.Series(res)
print(metrics.accuracy_score(test_Y, res1))

0.9579761904761904


In [15]:
# train regularized softmax regression model
from sklearn import linear_model
lm = OneVsRestClassifier(linear_model.LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs'))
lm.fit(train_data, Y)
res = lm.predict(test_data)

In [16]:
from sklearn import cross_validation
alphas = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10, 30, 100, 300, 1000, 3000]
scores = []
param = []
for alpha in alphas:
    lm = OneVsRestClassifier(linear_model.LogisticRegression(C = alpha, multi_class = 'multinomial', solver = 'lbfgs'))
    scores.append(cross_validation.cross_val_score(lm, train_data, Y, scoring="accuracy", cv = 10).mean())
    param.append([alpha])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

C:\Users\sanketn\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


   parameter     score
4      [0.1]  0.915167
5      [0.3]  0.915143
6      [1.0]  0.914357
7      [3.0]  0.913382
12    [1000]  0.912786
3     [0.03]  0.912738
8       [10]  0.912715
13    [3000]  0.912620
9       [30]  0.912596
10     [100]  0.912572
11     [300]  0.912548
2     [0.01]  0.908285
1    [0.003]  0.899237
0    [0.001]  0.887714


In [17]:
from sklearn import cross_validation
alphas = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10, 30, 100, 300, 1000, 3000]
scores = []
param = []
for alpha in alphas:
    lm = OneVsOneClassifier(linear_model.LogisticRegression(C = alpha, multi_class = 'multinomial', solver = 'lbfgs'))
    scores.append(cross_validation.cross_val_score(lm, train_data, Y, scoring="accuracy", cv = 10).mean())
    param.append([alpha])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

   parameter     score
4      [0.1]  0.938049
5      [0.3]  0.937786
3     [0.03]  0.935310
6      [1.0]  0.935214
7      [3.0]  0.932192
2     [0.01]  0.929428
8       [10]  0.927596
9       [30]  0.925120
10     [100]  0.924025
12    [1000]  0.923335
13    [3000]  0.923287
11     [300]  0.923286
1    [0.003]  0.919477
0    [0.001]  0.905310


In [76]:
# train regularized softmax regression model
from sklearn import linear_model
lm = OneVsOneClassifier(linear_model.LogisticRegression(C = 0.1, multi_class = 'multinomial', solver = 'lbfgs'))
lm.fit(train_data, Y)
res = lm.predict(test)

In [77]:
res1 = pd.Series(res)
print(metrics.accuracy_score(test_Y, res1))

0.9552380952380952


In [78]:
# train regularized softmax regression model
from sklearn import linear_model
lm = OneVsRestClassifier(linear_model.LogisticRegression(C = 0.1, multi_class = 'multinomial', solver = 'lbfgs'))
lm.fit(train_data, Y)
res1 = lm.predict(test)

In [79]:
res2 = pd.Series(res1)
print(metrics.accuracy_score(test_Y, res2))

0.9258333333333333


In [63]:
import numpy as np
import math
# RBF function
def rbf(data, centers, sigma):
    res = np.ndarray(shape = (len(data), len(centers)))
    i = 0
    for row in data:
        tmp = []
        for center in centers:
            tmp.append(math.exp(-1.0 * sum(np.square(row - center)) / (2 * sigma * sigma)))
        res[i, :] = tmp
        i = i + 1
    return res
# select k centers from data
def selectCenters(data, k):
    tmp = np.random.choice(len(data), k)
    return data[tmp,:]

In [85]:
# cross validation on regularized softmax regression
from sklearn import cross_validation
alphas = [0.1, 1.0, 10]
sigmas = [1.0, 2.0, 4.0]
Ks = [100, 400, 800, 1600]
scores = []
param = []
for K in Ks:
    centers = selectCenters(train_data.as_matrix(), K)
    for sigma in sigmas:
        rbfX = rbf(train_data.as_matrix(), centers, sigma)
        for a in alphas:
            lm = linear_model.LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', C = a)
            scores.append(cross_validation.cross_val_score(lm, rbfX, Y, scoring="accuracy", cv = 10).mean())
            param.append([K, sigma, a])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

           parameter     score
35   [1600, 4.0, 10]  0.954834
26    [800, 4.0, 10]  0.950810
34  [1600, 4.0, 1.0]  0.948572
17    [400, 4.0, 10]  0.942500
25   [800, 4.0, 1.0]  0.939787
16   [400, 4.0, 1.0]  0.928119
33  [1600, 4.0, 0.1]  0.919906
24   [800, 4.0, 0.1]  0.906977
8     [100, 4.0, 10]  0.899405
15   [400, 4.0, 0.1]  0.889404
7    [100, 4.0, 1.0]  0.874595
6    [100, 4.0, 0.1]  0.810261
32   [1600, 2.0, 10]  0.764737
23    [800, 2.0, 10]  0.719550
31  [1600, 2.0, 1.0]  0.715403
14    [400, 2.0, 10]  0.666048
22   [800, 2.0, 1.0]  0.644762
13   [400, 2.0, 1.0]  0.517572
5     [100, 2.0, 10]  0.490359
30  [1600, 2.0, 0.1]  0.473761
21   [800, 2.0, 0.1]  0.411332
4    [100, 2.0, 1.0]  0.362429
12   [400, 2.0, 0.1]  0.345238
3    [100, 2.0, 0.1]  0.241905
29   [1600, 1.0, 10]  0.190285
28  [1600, 1.0, 1.0]  0.186928
19   [800, 1.0, 1.0]  0.185761
20    [800, 1.0, 10]  0.183000
11    [400, 1.0, 10]  0.173429
2     [100, 1.0, 10]  0.168524
10   [400, 1.0, 1.0]  0.112857
1    [10

In [64]:
centers = selectCenters(train_data.as_matrix(), 1600)
rbfX = rbf(train_data.as_matrix(), centers, 4.0)
rbfTest = rbf(test.as_matrix(), centers, 4.0)
lm = linear_model.LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', C = 10)
lm.fit(rbfX, Y)
res = lm.predict(rbfTest)

In [71]:
from sklearn.decomposition import PCA
from sklearn import cross_validation
alphas = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10, 30, 100, 300, 1000, 3000]
Ks = [10, 100, 500]
scores = []
param = []
for K in Ks: 
    pca = PCA(n_components = K, svd_solver='arpack')
    pca.fit(train_data)
    pca_train = pca.transform(train_data)
    for alpha in alphas:
        lm = linear_model.LogisticRegression(C = alpha, multi_class = 'multinomial', solver = 'lbfgs')
        scores.append(cross_validation.cross_val_score(lm, pca_train, Y, scoring="accuracy", cv = 10).mean())
        param.append([K, alpha])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

       parameter     score
33    [500, 0.3]  0.915833
32    [500, 0.1]  0.915524
20    [100, 1.0]  0.914024
27   [100, 3000]  0.913952
18    [100, 0.1]  0.913952
25    [100, 300]  0.913929
21    [100, 3.0]  0.913881
24    [100, 100]  0.913857
26   [100, 1000]  0.913833
23     [100, 30]  0.913833
22     [100, 10]  0.913833
19    [100, 0.3]  0.913786
31   [500, 0.03]  0.913309
34    [500, 1.0]  0.912357
17   [100, 0.03]  0.910786
30   [500, 0.01]  0.906476
35    [500, 3.0]  0.905976
16   [100, 0.01]  0.904333
36     [500, 10]  0.898905
37     [500, 30]  0.895000
29  [500, 0.003]  0.894095
38    [500, 100]  0.893643
15  [100, 0.003]  0.892667
39    [500, 300]  0.892572
40   [500, 1000]  0.892381
41   [500, 3000]  0.892072
28  [500, 0.001]  0.877024
14  [100, 0.001]  0.876119
12    [10, 1000]  0.801428
11     [10, 300]  0.801428
10     [10, 100]  0.801428
9       [10, 30]  0.801428
13    [10, 3000]  0.801405
8       [10, 10]  0.801357
7      [10, 3.0]  0.801357
6      [10, 1.0]  0.801286
5

In [65]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 500, svd_solver='arpack')
pca.fit(train_data)
pca_train = pca.transform(train_data)
pca_test = pca.transform(test)
lm = linear_model.LogisticRegression(C = 0.3, multi_class = 'multinomial', solver = 'lbfgs')
lm.fit(pca_train, Y)
res = lm.predict(pca_test)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
nigs = [1, 10, 20]
scores = []
param = []
for n in nigs:
    neigh = KNeighborsClassifier(n_neighbors = n)
    scores.append(cross_validation.cross_val_score(neigh, rbfX, Y, scoring="accuracy", cv = 10).mean())
    param.append([n])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

  parameter     score
1      [10]  0.956643
0       [1]  0.954523
2      [20]  0.953048


In [66]:
from sklearn.neighbors import KNeighborsClassifier
neigh1 = KNeighborsClassifier(n_neighbors = 10)
neigh1.fit(rbfX, Y)
res1 = neigh1.predict(rbfTest)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
nigs = [1, 10, 20]
scores = []
param = []
for n in nigs:
    neigh = KNeighborsClassifier(n_neighbors = n)
    scores.append(cross_validation.cross_val_score(neigh, pca_train, Y, scoring="accuracy", cv = 10).mean())
    param.append([n])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

  parameter     score
0       [1]  0.967405
1      [10]  0.964523
2      [20]  0.958333


In [67]:
from sklearn.neighbors import KNeighborsClassifier
neigh2 = KNeighborsClassifier(n_neighbors = 1)
neigh2.fit(pca_train, Y)
res = neigh2.predict(pca_test)

In [68]:
res[:10]

array([1, 0, 1, 4, 0, 0, 7, 3, 5, 3], dtype=int64)

In [70]:
res1 = pd.Series(res)
print(metrics.accuracy_score(test_Y, res1))

1.0


In [69]:
res1[:10]

array([1, 0, 1, 4, 0, 0, 7, 5, 5, 3], dtype=int64)

In [71]:
res2 = pd.Series(res1)
print(metrics.accuracy_score(test_Y, res2))

1.0


In [87]:
# save predictions
sample_data = pd.read_csv("./data/sample_submission.csv")
sample_data['Label'] = res
sample_data.to_csv('./prediction.csv', index = False)